# Geoqueries en Mongo

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#¿Qué-podemos-hacer-con-las-geoqueries?" data-toc-modified-id="¿Qué-podemos-hacer-con-las-geoqueries?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>¿Qué podemos hacer con las geoqueries?</a></span></li><li><span><a href="#Generamos-la-conexión-con-Mongo" data-toc-modified-id="Generamos-la-conexión-con-Mongo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generamos la conexión con Mongo</a></span></li><li><span><a href="#Importar-colecciones" data-toc-modified-id="Importar-colecciones-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importar colecciones</a></span></li><li><span><a href="#Crear-indexes" data-toc-modified-id="Crear-indexes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Crear indexes</a></span></li><li><span><a href="#Generamos-lasconexionescon-la-base-de-datos" data-toc-modified-id="Generamos-lasconexionescon-la-base-de-datos-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generamos lasconexionescon la base de datos</a></span></li><li><span><a href="#Empezamos-con-las-Geoqueries" data-toc-modified-id="Empezamos-con-las-Geoqueries-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Empezamos con las Geoqueries</a></span></li><li><span><a href="#Intersects" data-toc-modified-id="Intersects-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Intersects</a></span></li><li><span><a href="#GeoWithin" data-toc-modified-id="GeoWithin-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>GeoWithin</a></span></li><li><span><a href="#Near" data-toc-modified-id="Near-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Near</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Extra--->-Revisad-$geoNear" data-toc-modified-id="Extra--->-Revisad-$geoNear-9.0.0.1"><span class="toc-item-num">9.0.0.1&nbsp;&nbsp;</span>Extra --&gt; Revisad $geoNear</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Extra,-extra!!-mis-datos-extraídos-de-Foursquare-en-Mongo" data-toc-modified-id="Extra,-extra!!-mis-datos-extraídos-de-Foursquare-en-Mongo-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Extra, extra!! mis datos extraídos de Foursquare en Mongo</a></span><ul class="toc-item"><li><span><a href="#Voy-a-traerme-todas-las-fruterías-de-Madrid" data-toc-modified-id="Voy-a-traerme-todas-las-fruterías-de-Madrid-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Voy a traerme todas las fruterías de Madrid</a></span></li></ul></li><li><span><a href="#Vamos-a-buscar-a-cuántos-km-están-las-fruterías-de-mi-casa" data-toc-modified-id="Vamos-a-buscar-a-cuántos-km-están-las-fruterías-de-mi-casa-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Vamos a buscar a cuántos km están las fruterías de mi casa</a></span></li></ul></div>

## ¿Qué podemos hacer con las geoqueries?
Con las geoquerías podemos responder a preguntas como las siguientes
 * ¿Dónde están las farmacias más cercanas a mi ubicación?
 * ¿Qué restaurantes hay en esta poligonal?

Cada punto de la Tierra se caracteriza por dos números:
 * Longitud: ángulo respecto al meridiano de Greenwich. Va de -180° (antiGreenwich) a +180° (también antiGreenwich)
 * Latitud: ángulo con respecto al ecuador. Va de -90° (sur) a +90° (norte)

![latierra](../images/latlon.gif)

## Generamos la conexión con Mongo

In [1]:
from pymongo import MongoClient
conn = MongoClient("localhost:27017")
db = conn.get_database("ironhack")

## Importar colecciones     
Sigamos la documentación oficial de Mongo: [Mongo Geo Example](https://docs.mongodb.com/manual/tutorial/geospatial-tutorial/)
Creamos estas dos colecciones nuevas:
 * `restaurants2`
 * `neighborhoods`

## Crear indexes
La indexación geoespacial de MongoDB permite   
ejecutar eficientemente **consultas espaciales** sobre una colección que contiene formas y puntos geoespaciales.
Vamos a crear un índice geográfico para que Mongo sepa que queremos hacer geo-consultas en esta colección.  
Esto sólo debe hacerse una vez, y ahora la colección de Mongo está correctamente indexada para siempre.

In [2]:
from pymongo import GEOSPHERE

In [3]:
# Creamos el  index 2dsphere desde pymongo
db.restaurants2.create_index([("location", GEOSPHERE)])

'location_2dsphere'

In [4]:
db.restaurants2.find_one({})

Es importante indicarle el campo donde  tenemo tanto el tipo de elemento (polígonos o  puntos (point)) como las coordenadas.

In [5]:
db.neighborhoods.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [6]:
#db.neighborhoods.find_one({})

## Generamos lasconexionescon la base de datos

In [7]:
n = db.get_collection("neighborhoods")
r = db.get_collection("restaurants2")

## Empezamos con las Geoqueries   
¿En qué barrio estoy?      
¿Qué elementos **intersecan** el elemento dado?

## Intersects

Necesitamos tener los datos  con tipo polígono en este caso tenemos una colección con barrios donde cada barrio  tiene dentro  el polígono que lo delimita. Y cada barrio es un documento, por eso cuando  intersecta me devuelve el documento y yo veo  que es el barrio  en concreto.

In [8]:
coordenadas = [-73.93, 40.82]

In [9]:
coord_buscar = {"type":"Point", "coordinates": coordenadas}

In [10]:
proj = {"_id":0, "name": 1}

In [11]:
n.find_one().keys()

AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
n.find_one({"geometry": {"$geoIntersects": {"$geometry":coord_buscar}}},proj)

## GeoWithin

¿Qué elementos están **contenidos** en el elemento dado?     
¿Qué restaurantes hay en este barrio (polígono)?

In [ ]:
# Vemos la estructura de los documentos sacando uno y viendo sus keys
n.find_one({}).keys()

In [ ]:
#bUSCAMOS UN barrio en  concreto
barrio = n.find_one({"name": "West Concourse"})
#barrio

In [ ]:
#Extraigo las coordenadas
geo = barrio.get("geometry")
#geo

In [ ]:
#Saco las keys de un documento d e la coleccción  de restaurantes
r.find_one({}).keys()

In [ ]:
query = r.find({"location": {"$geoWithin": {"$geometry": geo}}})

In [ ]:
# ER FIRTRO
filtro = {"location": {"$geoWithin": {"$geometry": geo}}}

In [ ]:
import pandas as pd
df = pd.DataFrame(list(query))
df.head()

## Near

¿Qué elementos se encuentran a una distancia de una geometría determinada?     
`$nearSphere`/`$near` y `$maxDistance` nos ayudan a encontrar todos los elementos dentro de `maxDistance` metros de la geometría deseada, **ordenados** de más cercano a más lejano.

In [ ]:
#  Mi posición, voy a buscar restaurantes cercanos a  ESTE punto
coordenadas2 = [-73.93, 40.82]
coord_point = {"type":"Point", "coordinates": coordenadas2}

In [ ]:
# Km que quiero  caminar
metros = 3000

In [ ]:
query = {"location": {"$near": {"$geometry": coord_point,"$minDistance": 500  , "$maxDistance": metros}}}

In [ ]:
query_final = r.find(query,proj)

In [ ]:
df = pd.DataFrame(list(query_final))

In [ ]:
df.head()

##### Extra --> Revisad $geoNear 
https://docs.mongodb.com/manual/reference/operator/aggregation/geoNear/

## Extra, extra!! mis datos extraídos de Foursquare en Mongo

### Voy a traerme todas las fruterías de Madrid

In [13]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

True

In [14]:
tok1= os.getenv("tok1")
tok2= os.getenv("tok2")

In [16]:
madrid = {'type': 'Point', 'coordinates': [40.42955,-3.6793]}

In [17]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [18]:
parametros = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{madrid.get('coordinates')[0]},{madrid.get('coordinates')[1]}",
    "query": "frutería", 
    "limit": 100    
}

In [ ]:
resp = requests.get(url= url_query, params = parametros).json()

In [ ]:
#resp

In [ ]:
data = resp.get("response").get("groups")[0].get("items")

In [ ]:
from functools import reduce
import operator

In [ ]:
# Mis fuentes --stackoverflow
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [ ]:
mapa_nombre =  ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]

In [ ]:
lista_frutas = []
for dic in data:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    lista_frutas.append(paralista)

In [ ]:
df = pd.DataFrame(lista_frutas)
df.head()

Vamos a manipular los datos para tener esta misma estructura en tipo point y poder hacer un index en Mongo para realizar geoqueries

In [ ]:
lista_frutas[0]

In [ ]:
documentos = []
for diccionario in lista_frutas:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]}
        
    }
    documentos.append(temporal)

In [ ]:
documentos[0]

In [20]:
df = pd.DataFrame(documentos)
df.head()

NameError: name 'documentos' is not defined

In [19]:
df.to_json("lasfruterias.json", orient="records")

NameError: name 'df' is not defined

## Vamos a buscar a cuántos km están las fruterías de mi casa

In [ ]:
fruterias = db.get_collection("misfruterias")
fruterias.create_index([("geometry", GEOSPHERE)])

In [ ]:
fruterias.find_one({})

In [ ]:
lavapi = [40.408808, -3.7034796]
lavapi_point = {"type": "Point",  "coordinates": lavapi}

In [ ]:
lavapi_point

In [ ]:
consulta = {"location": {"$near": {"$geometry": lavapi_point, "$maxDistance": 2000}}}

In [ ]:
fru = list(fruterias.find(consulta))

In [ ]:
df = pd.DataFrame(fru)
df.head()